# Converting CSV annotations (with or without missing columns) to COCO JSON format.
### Tutorial by Amirhesam Yazdi
The minumum requirements is having the filenames for the images, and bounding boxes, similar to the Wildfire detection open-fire-tech dataset (used in [this paper](https://doi.org/10.3390/rs12010166) by Lawrence Berkeley National Lab & Fireball LLC).

|    |   MinX |   MinY |   MaxX |   MaxY | Filename                             |
|---:|-------:|-------:|-------:|-------:|:-------------------------------------|
|  0 |    547 |    974 |    608 |   1007 | bh-w-mobo-c__2019-06-10T13;26;16.jpg |
|  1 |    522 |    971 |    612 |   1011 | bh-w-mobo-c__2019-06-10T13;27;16.jpg |
|  2 |    537 |    949 |    606 |   1004 | bh-w-mobo-c__2019-06-10T13;28;16.jpg |
|  3 |    499 |    918 |    604 |   1004 | bh-w-mobo-c__2019-06-10T13;29;16.jpg |
|  4 |    472 |    894 |    606 |   1004 | bh-w-mobo-c__2019-06-10T13;30;16.jpg |

The other columns (class, width, height, ...) can be derived based on context and images. You can edit the csv file directly and add placeholder columns and then load it with pandas; or you can add columns after loading with pandas.

In [1]:
import numpy as np
import json
import pandas as pd
import cv2

In [2]:
path = 'FuegoDataset/annot_fuego.csv'
save_json_path='FuegoDataset/fuegoConvert.json'
img_path='FuegoDataset/1_smoke/'

In [3]:
data = pd.read_csv(path)
df = data.copy()

If your csv file doesn't include annotations for all the classes in your problem, then those categories will not be created automatically.
There are 2 options:
> 1. add an annotation entry for each class that is not represented.
> 2. (preferred) make sure to manually edit the categories section of the converted json file to include the missing categories. 

```
data.tail()
```
|      | filename                             | class      |   width |   height |   xmin |   ymin |   xmax |   ymax |
|-----:|:-------------------------------------|:-----------|--------:|---------:|-------:|-------:|-------:|-------:|
| 1737 | wc-s-mobo-c__2019-09-24T15;26;22.jpg | smoke      |     nan |      nan |     75 |    854 |    296 |    990 |
| 1738 | wc-s-mobo-c__2019-09-24T15;27;22.jpg | smoke      |     nan |      nan |     73 |    863 |    267 |    988 |
| 1739 | wc-s-mobo-c__2019-09-24T15;27;22.jpg | fire       |     nan |      nan |     73 |    863 |    267 |    988 |
| 1740 | wc-s-mobo-c__2019-09-24T15;27;22.jpg | flame      |     nan |      nan |     73 |    863 |    267 |    988 |
| 1741 | wc-s-mobo-c__2019-09-24T15;27;22.jpg | NightSmoke |     nan |      nan |     73 |    863 |    267 |    988 |

In [4]:
images = []
categories = []
annotations = []

category = {}
category["supercategory"] = 'none'
category["id"] = 0
category["name"] = 'None'
categories.append(category)

In [5]:
data['fileid'] = data['filename'].astype('category').cat.codes
data['categoryid']= pd.Categorical(data['class'],categories=['smoke','fire','flame','NightSmoke']).codes
data['categoryid'] = data['categoryid']+1 # + 1 is because we are keeping 0 for N/A and ommiting it from Categories
data['annid'] = data.index + 1

```
data.tail()
```
|      | filename                             | class      |   width |   height |   xmin |   ymin |   xmax |   ymax |   fileid |   categoryid |   annid |
|-----:|:-------------------------------------|:-----------|--------:|---------:|-------:|-------:|-------:|-------:|---------:|-------------:|--------:|
| 1737 | wc-s-mobo-c__2019-09-24T15;26;22.jpg | smoke      |     nan |      nan |     75 |    854 |    296 |    990 |     1645 |            1 |    1738 |
| 1738 | wc-s-mobo-c__2019-09-24T15;27;22.jpg | smoke      |     nan |      nan |     73 |    863 |    267 |    988 |     1646 |            1 |    1739 |
| 1739 | wc-s-mobo-c__2019-09-24T15;27;22.jpg | fire       |     nan |      nan |     73 |    863 |    267 |    988 |     1646 |            2 |    1740 |
| 1740 | wc-s-mobo-c__2019-09-24T15;27;22.jpg | flame      |     nan |      nan |     73 |    863 |    267 |    988 |     1646 |            3 |    1741 |
| 1741 | wc-s-mobo-c__2019-09-24T15;27;22.jpg | NightSmoke |     nan |      nan |     73 |    863 |    267 |    988 |     1646 |            4 |    1742 |

### getting image pixel dimension (width and height) from the images

In [6]:
for row in data.itertuples():
    temp_path = img_path + row.filename
    image = cv2.imread(temp_path)
    data.loc[row.Index,'width'], data.loc[row.Index,'height'],_ = image.shape
    print(row.Index, '\t', row.filename, end='\r')

```
data.sample()
```
|      | filename                                  | class   |   width |   height |   xmin |   ymin |   xmax |   ymax |   fileid |   categoryid |   annid |
|-----:|:------------------------------------------|:--------|--------:|---------:|-------:|-------:|-------:|-------:|---------:|-------------:|--------:|
| 1347 | pi-s-mobo-c__2019-08-26T12;02;08.jpg      | smoke   |    1536 |     2048 |   1231 |    730 |   1264 |    760 |      875 |            1 |    1348 |
|  515 | smer-tcs9-mobo-c__2019-10-03T12;35;00.jpg | smoke   |    2048 |     3072 |   1860 |    729 |   1980 |    814 |     1484 |            1 |     516 |
| 1471 | smer-tcs9-mobo-c__2019-08-26T16;22;57.jpg | smoke   |    2048 |     3072 |   1753 |    794 |   1811 |    831 |     1403 |            1 |    1472 |
|  505 | rm-w-mobo-c__2019-10-03T13;40;09.jpg      | smoke   |    1536 |     2048 |    866 |    761 |    944 |    834 |     1199 |            1 |     506 |
| 1732 | wc-s-mobo-c__2019-09-24T15;17;22.jpg      | smoke   |    2048 |     3072 |    100 |    850 |    382 |    975 |     1640 |            1 |    1733 |
| 1683 | ml-w-mobo-c__2019-09-24T15;12;17.jpg      | smoke   |    1536 |     2048 |    699 |    833 |    755 |    882 |      606 |            1 |    1684 |
|   83 | om-e-mobo-c__2019-10-01T11;20;37.jpg      | smoke   |    2048 |     3072 |   2557 |   1040 |   2731 |   1206 |      687 |            1 |      84 |
| 1552 | rm-w-mobo-c__2019-08-29T11;55;06.jpg      | smoke   |    1536 |     2048 |   1065 |    782 |   1139 |    819 |     1081 |            1 |    1553 |
| 1729 | wc-s-mobo-c__2019-09-24T15;10;24.jpg      | smoke   |    2048 |     3072 |    127 |    846 |    275 |    994 |     1637 |            1 |    1730 |
|  938 | bh-w-mobo-c__2019-06-10T13;24;16.jpg      | smoke   |    1536 |     2048 |    567 |    970 |    631 |   1015 |       39 |            1 |     939 |

#### functions to retrieve tabular information to populat each section of the json

In [7]:
def image(row):
    image = {}
    image["height"] = row.height
    image["width"] = row.width
    image["id"] = row.fileid
    image["file_name"] = row.filename
    return image

def category(row):
    category = {}
    category["supercategory"] = 'None'
    category["id"] = row.categoryid
    category["name"] = row[2]
    return category

def annotation(row):
    annotation = {}
    area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
    annotation["segmentation"] = []
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = row.fileid

    annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

    annotation["category_id"] = row.categoryid
    annotation["id"] = row.annid
    return annotation

In [8]:
# populate annotations
for row in data.itertuples():
    annotations.append(annotation(row))

Last 5 annoatations
```
annotations[-5:]
```
[{'segmentation': [],
  'iscrowd': 0,
  'area': 30056,
  'image_id': 1645,
  'bbox': [75, 854, 221, 136],
  'category_id': 1,
  'id': 1738},
 {'segmentation': [],
  'iscrowd': 0,
  'area': 24250,
  'image_id': 1646,
  'bbox': [73, 863, 194, 125],
  'category_id': 1,
  'id': 1739},
 {'segmentation': [],
  'iscrowd': 0,
  'area': 24250,
  'image_id': 1646,
  'bbox': [73, 863, 194, 125],
  'category_id': 2,
  'id': 1740},
 {'segmentation': [],
  'iscrowd': 0,
  'area': 24250,
  'image_id': 1646,
  'bbox': [73, 863, 194, 125],
  'category_id': 3,
  'id': 1741},
 {'segmentation': [],
  'iscrowd': 0,
  'area': 24250,
  'image_id': 1646,
  'bbox': [73, 863, 194, 125],
  'category_id': 4,
  'id': 1742}]

sorting the images based on their fileid and removing duplicates to then extract list of images for the images section of COCO json format

In [9]:
imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
for row in imagedf.itertuples():
    images.append(image(row))

Keep only one row from each category and then extract the category ids for the category section

In [10]:
catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
for row in catdf.itertuples():
    categories.append(category(row))

Combining it all!

In [11]:
data_coco = {}
data_coco["images"] = images
data_coco["categories"] = categories
data_coco["annotations"] = annotations

Saving the converted file in the specified path.

In [12]:
json.dump(data_coco, open(save_json_path, "w"), indent=4)

#### NOTE: You might need to remove category 0 from the converted annotations. 
> DETR doesn't explicitly define "N/A" categories associated with no-object. They are skipped in the category section but counted in number of class. 
> It is also possible to have more than one N/A class by skipping more categories and then counting them in the model. Refer to DETR hands-on colab to see the full list of classes. There you will notice that roughly for each 10 object classes, there is a N/A class. But these classes are not explicitly defined in Categories in COCO dataset. 